In [1]:
import sys
import os

path = os.path.dirname(os.path.abspath("__file__"))
sys.path.insert(0, path + '/../..')

import base64
import IPython
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory

from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['Pendulum-v0']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator
import reinforcement_learning.environments
from policies.saved_policy import SavedTFPolicy
from policies.epsilon_mimic import EpsilonMimicPolicy
from policies.latent_policy import LatentPolicyOverRealStateAndActionSpaces

In [2]:
py_env = suite_gym.load('Pendulum-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3,), dtype=tf.float32, name='observation', minimum=array([-1., -1., -8.], dtype=float32), maximum=array([1., 1., 8.], dtype=float32)))

In [4]:
tf_env = tf_py_environment.TFPyEnvironment(py_env)

sac_policy_dir = '../saves/Pendulum-v0/policy'
policy = tf.compat.v2.saved_model.load(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=10,
                                            observers=[
                                                # display_safe_labeling,
                                                lambda _: py_env.render(mode='human'),
                                            ]).run()

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.31054267, -0.95055944, -0.18270244]], dtype=float32)>),
 ())

In [5]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    # "../../saves/Pendulum-v0/models/vae_LS12_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA3_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step140000/eval_elbo-3.784"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA5_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step270000/eval_elbo-1.704"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/permissive_variance_policy-multiplier=10.0/action_discretizer/LA5_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step70000/eval_elbo-7.265"
    # "../../saves/Pendulum-v0/models/vae_LS12_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.95-0.90_1e-06-2e-06/policy/action_discretizer/LA3_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step1010000/eval_elbo11.174"
    # '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.95-0.90_1e-06-2e-06/policy/action_discretizer/LA3_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step1010000/eval_elbo-2.888'
    # '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.70-0.57_2e-06-1e-06_seed=290321/policy/action_discretizer/LA3_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.50-0.33_2e-06-1e-06_params=full_vae_optimization-relaxed_state_encoding/step360000/eval_elbo4.516'
    #'../../saves/Pendulum-v0/models/vae_LS12_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e-06-2e-06_seed=20421/policy/action_discretizer/LA3_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization-relaxed_state_encoding/base',
    #step=1830000
    '../../saves/Pendulum-v0/models/vae_LS14_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e'
    '-06-2e'
    '-06_seed=20210521/policy/action_discretizer/LA3_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0'
    '.50-0.33_1e-06-2e-06_PER-priority_exponent=0.99-WIS_exponent=0.4-WIS_growth_rate=1e'
    '-05loss_based_priorities_params=one_output_per_action-full_vae_optimization-relaxed_state_encoding/base',
    step=310000
)
print("VAE MDP loaded")

vae_mdp.eval_policy(eval_env=py_env, labeling_function=labeling_function, num_eval_episodes=20, render=True)

VAE MDP loaded
NaN values occurred in the environment while the driver was running:
ValueError('Received a mix of batched and unbatched Tensors, or Tensors are not compatible with Specs.  num_outer_dims: 1.\nSaw tensor_shapes:\n   TimeStep(step_type=TensorShape([1]), reward=TensorShape([1]), discount=TensorShape([1]), observation=TensorShape([1, 12]))\nAnd spec_shapes:\n   TimeStep(step_type=TensorShape([]), reward=TensorShape([]), discount=TensorShape([]), observation=TensorShape([14]))')
eval policy -inf


-inf

In [4]:
import imageio
import IPython

def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

def create_policy_eval_video(policy, py_env, tf_env, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = tf_env.reset()
      video.append_data(py_env.render(mode='rgb_array'))
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render(mode='rgb_array'))
  return embed_mp4(filename)

In [6]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    '../../saves/PendulumRandomInit-v0/models/vae_LS13_MC1_ER10.0-decay=7.5e-05-min='
    '0_KLA0.0-growth=7.5e-05_TD0.67-0.50_1e-06-2e-06_seed=20210619/policy/action_discretizer/'
    'LA3_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=7.5e-05_TD0.50-0.33_1e-06-2e-06_PER-P_exponent='
    '0.3-WIS_exponent=0.4-WIS_growth=1e-05_loss_based_epsilon_greedy=0.4-decay=4e-06_params='
    'full_vae_optimization-relaxed_state_encoding/base',
    step=400000
)
print("VAE MDP loaded")

vae_mdp.eval_policy(eval_env=suite_gym.load('PendulumRandomInit-v0'), labeling_function=labeling_function, num_eval_episodes=20, render=True)

VAE MDP loaded


eval policy -278.200256


<tf.Tensor: shape=(), dtype=float32, numpy=-278.20026>

In [18]:
py_env = suite_gym.load('PendulumRandomInit-v0')
tf_env = tf_py_environment.TFPyEnvironment(py_env)

create_policy_eval_video(
    policy=SavedTFPolicy('../saves/Pendulum-v0/policy'),
    py_env=py_env,
    tf_env=tf_env,
    filename='Pendulum-v0_RL_policy',
    num_episodes=20)

In [5]:
py_env = suite_gym.load('PendulumRandomInit-v0')
tf_env = tf_py_environment.TFPyEnvironment(py_env)

create_policy_eval_video(
    policy=vae_mdp.get_latent_policy(),
    py_env=py_env,
    tf_env=vae_mdp.wrap_tf_environment(tf_env, labeling_function),
    filename='Pendulum-v0_latent_policy_new',
    num_episodes=20)

In [2]:
py_env = suite_gym.load('PendulumRandomInit-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3,), dtype=tf.float32, name='observation', minimum=array([-1., -1., -8.], dtype=float32), maximum=array([1., 1., 8.], dtype=float32)))

In [3]:
n = 0
def display_safe_labeling(time_step):
    global n
    if n % 10 == 0:
        input(str(time_step.observation) + ' ' +
              str(labeling_functions['PendulumRandomInit-v0'](time_step.observation)))
    n += 1

reward_metric = tf_metrics.AverageReturnMetric()
sac_policy_dir = '../saves/PendulumRandomInit-v0/sac_policy/'
policy = tf.compat.v2.saved_model.load(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=20,
                                            observers=[
                                                lambda _: py_env.render(mode='human'),
                                                reward_metric,
                                                # display_safe_labeling,
                                            ]).run()
reward_metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=-84.20418>

In [2]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    '../../saves/PendulumRandomInit-v0/models/vae_LS13_MC1_ER10.0-decay=7.5e-05-min='
    '0_KLA0.0-growth=7.5e-05_TD0.67-0.50_1e-06-2e-06_seed=20210621/sac_policy/'
    'action_discretizer/LA3_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=7.5e-05_TD0.50-0.33_1e-06-2e-06_'
    'PER-P_exponent=0.3-WIS_exponent=0.4-WIS_growth=1e-05_loss_based_epsilon_greedy=0.5-decay'
    '=1e-05_params=full_vae_optimization-relaxed_state_encoding/base',
    step=220000
)
print("VAE MDP loaded")

VAE MDP loaded


In [11]:
vae_mdp.eval_policy(eval_env=py_env, labeling_function=labeling_function, num_eval_episodes=20, render=True)

'LA3_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=7.5e-05_TD0.50-0.33_1e-06-2e-06_PER-P_exponent=0.3-WIS_exponent=0.4-WIS_growth=1e-05_loss_based_epsilon_greedy=0.5-decay=1e-05_params=full_vae_optimization-relaxed_state_encoding/base'